In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os

# path = "C:\\KIMUJUNG\\team_project\\team_project3\\pdf_folder"
path = "D:\\study\\sknetworks\\team_project\\pdf_folder"
file_list = os.listdir(path)

print ("file_list: {}".format(file_list))

file_list: ['개인용_let click 개인용자동차보험(TM).pdf', '개인용_let click 개인용자동차보험(인터넷).pdf', '개인용_let way 개인용자동차보험.pdf', '공동인수_개인용 자동차보험(공동).pdf', '공동인수_업무용 자동차보험(공동).pdf', '공동인수_영업용 자동차보험(공동).pdf', '공동인수_이륜자동차보험(공동).pdf', '기타_CREW 원데이자동차보험.pdf', '기타_let click 배달플랫폼자동차보험(인터넷).pdf', '기타_let way 배달플랫폼자동차보험.pdf', '기타_let way 자동차취급업자종합보험.pdf', '농기계_let way 농기계보험.pdf', '상해,질병_(무) let care 간병보험Ⅱ(2501).pdf', '상해,질병_(무) let care 노후실손의료보험Ⅱ(2501).pdf', '상해,질병_(무) let care 단체보험(2501).pdf', '상해,질병_(무) let care 상해보험(2404) 1종.pdf', '상해,질병_(무) let care 상해보험(2404) 2종.pdf', '상해,질병_(무) let care 실손의료보험Ⅳ(2501).pdf', '상해,질병_(무) let care 실손의료보험Ⅳ(개인 단체연계용)(2501).pdf', '상해,질병_(무) let care 실손의료보험Ⅳ(계약전환용)(2501).pdf', '상해,질병_(무) let care 유병력자실손의료보험(2406).pdf', '상해,질병_(무) let care 치아보험(2404).pdf', '상해,질병_(무) let click 실손의료보험Ⅳ(계약전환용)(2501).pdf', '상해,질병_(무) let play 1540종합보험(10플러스)(2501).pdf', '상해,질병_(무) let play 자녀보험(도담도담)(2501).pdf', '상해,질병_(무) let play 종합건강보험(토닥토닥)(2501).pdf', '상해,질병_(무) let simple 3N5 간편맞춤 건강보험(세만기형)(2501)

In [ ]:
# 1️⃣ FAISS 저장 경로
faiss_index_path = "./faiss_index.bin"
metadata_path = "./documents.pkl"

# 2️⃣ PDF 문서 로드
for file in file_list[:3]:
    loader = PyPDFLoader(path + "\\" + file)
    documents = loader.load()

    # 3️⃣ 문서 전처리
    for doc in documents:
        doc.page_content = doc.page_content.replace("\n", " ").replace("  ", " ")
        doc.metadata["source"] = file


    # 1️⃣ 문서 스플리팅
    # splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)
    splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=1000)
    documents = splitter.split_documents(documents)

    # 2️⃣ 스플릿된 문서로 벡터 생성
    batch_size = 16
    texts = [doc.page_content for doc in documents]
    embeddings = []

    for i in range(0, len(texts), batch_size):
        batch = texts[i : i + batch_size]
        batch_embeddings = get_embeddings(batch)  # 🔥 배치 단위로 임베딩 생성
        embeddings.extend(batch_embeddings)

    # 3️⃣ FAISS 인덱스에 추가
    embedding_dim = len(embeddings[0])
    index = faiss.IndexFlatL2(embedding_dim)
    index.add(np.array(embeddings, dtype=np.float32))

    # 4️⃣ FAISS 인덱스 & 문서 저장
    faiss.write_index(index, "./faiss_index.bin")
    with open("./documents.pkl", "ab") as f:
        pickle.dump(documents, f)

    print(file)


c:\Users\USER\anaconda3\envs\vectordb_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


개인용_let click 개인용자동차보험(TM).pdf
개인용_let click 개인용자동차보험(인터넷).pdf
개인용_let way 개인용자동차보험.pdf


In [15]:
# 1️⃣ 저장된 FAISS 인덱스 로드
index = faiss.read_index("./faiss_index.bin")

# 2️⃣ 문서 정보 로드
with open("./documents.pkl", "rb") as f:
    documents = pickle.load(f)

# 3️⃣ 검색 수행
# query = " 보험금의 종류 및 한도에 대해 설명해줘?"
query = "의무보험 미가입에 따른 불이익을 알려줘"
query_embedding = get_embeddings([query])[0]  # 쿼리 임베딩
query_embedding = np.array([query_embedding], dtype=np.float32)

D, I = index.search(query_embedding, k=5)  # 가장 유사한 5개 검색
context = []

# 4️⃣ 검색 결과 출력
for idx in I[0]:
    context.append(documents[idx].page_content)
    print(f"🔹 문서 {idx}: {documents[idx].page_content[:200]}")


🔹 문서 80: 않는 경우 해당 계약을 해지할 수 있습니다. ④ 제1항 및 제3항에 따라 계약이 해지된 경우 회사는 제54조(보험료의 환급) 제3항 제1호에 따른 보험료를 계약자에게 지 급합니다. ⑤ 계약자는 제1항에 따른 제척기간１２８)에도 불구하고 민법 등 관계 법령에서 정하는 바에 따라 법률상의 권리를 행사할 수 있습니다. 제53조(보험회사의 보험계약 해지) ① 보험
🔹 문서 84: 관하여 필요한 조사를 하거나 보험계약자 또는 피보험자에게 필요한 설명 또는 증명을 요구 할 수 있습니다. 이 경우 보험계약자, 피보험자 또는 이들의 대리인은 이러한 조사 또는 요구에 협력하여야 합니다. 제58조(예금보험기금에 의한 보험금 등의 지급보장) 보험회사가 파산 등으로 인하여 보험금 등을 지급하지 못할 경우에는 예금자보호법(*1)에서 정하는 바에 따
🔹 문서 6: 보 통 약 관 특 별 약 관 교 통 사 고 발 생 시 대 처 요 령 관 련 법 령 알 기 쉬 운 자 동 차 보 험 이 야 기 개인용자동차보험 7 1 의무보험의 의미 ‘의무보험’은 자동차 사고 피해자가 최소한의 보상을 받을 수 있도록 자동차손해 배상보장법에 의거하여 국가에서 법적으로 가입을 의무화한 보험입니다. 대인배상I은 최대 1억 5천만원, 대물배상은 최
🔹 문서 69: 한도(동일한 사고로 이미 지급한 보험금이나 가지급금이 있는 경우에는 그 금액을 공제한 금액. 이하 같음) 내에서 제1항의 절차에 협조하거나 대행합니다. ③  보험회사가 제1항의 절차에 협조하거나 대행하는 경우에는 피보험자는 보험회사의 요청에 따라 협력해야 합니다. 피보 험자가 정당한 이유 없이 협력하지 않는 경우 그로 인하여 늘어난 손해에 대하여는 보상하지
🔹 문서 8: 보 통 약 관 특 별 약 관 교 통 사 고 발 생 시 대 처 요 령 관 련 법 령 알 기 쉬 운 자 동 차 보 험 이 야 기 개인용자동차보험 9 1 청약내용 확인 관련 보험기간 중 계약 관련 사항이 변경된 경우 계약 담당자나  고객 콜센터(1588-3344, 1600-343

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Create a FAISS VectorStore
vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings() # deployment="text-embedding-ada-002"
)

# Create a Retriever
retriever = vectorstore.as_retriever()

system_message = "당신은 보험 상담 ASSISTANT입니다. 보험 약관인 context 기반으로 query에 대해 답변해 주세요."
prompt_text = f"[SYSTEM]: {system_message}\n[USER]: {query}\n[context]: {context}\n[ASSISTANT]"

prompt = ChatPromptTemplate.from_template(prompt_text)
model = ChatOpenAI() # model_name='gpt-3.5-turbo'

# Chaining a pipeline
retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# Run retrieva_chain
retrieval_chain.invoke("where did harrison work?")


In [17]:
import torch
from transformers import PhiForCausalLM, AutoModelForCausalLM, AutoTokenizer

torch.set_default_device("cpu")

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained("daekeun-ml/phi-2-ko-v0.1", torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained("daekeun-ml/phi-2-ko-v0.1", trust_remote_code=True)

system_message = "당신은 보험 상담 ASSISTANT입니다. 보험 약관인 context 기반으로 query에 대해 답변해 주세요."
prompt = f"[SYSTEM]: {system_message}\n[USER]: {query}\n[context]: {context}\n[ASSISTANT]"

inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_new_tokens=512)
text = tokenizer.batch_decode(outputs)[0]
print(text)


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  5.64it/s]
Token indices sequence length is longer than the specified maximum sequence length for this model (3056 > 2048). Running this sequence through the model will result in indexing errors
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[SYSTEM]: 당신은 보험 상담 ASSISTANT입니다. 보험 약관인 context 기반으로 query에 대해 답변해 주세요.
[USER]: 의무보험 미가입에 따른 불이익을 알려줘
[context]: ['않는 경우 해당 계약을 해지할 수 있습니다. ④ 제1항 및 제3항에 따라 계약이 해지된 경우 회사는 제54조(보험료의 환급) 제3항 제1호에 따른 보험료를 계약자에게 지 급합니다. ⑤ 계약자는 제1항에 따른 제척기간１２８)에도 불구하고 민법 등 관계 법령에서 정하는 바에 따라 법률상의 권리를 행사할 수 있습니다. 제53조(보험회사의 보험계약 해지) ① 보험회사는 다음 중 어느 하나에 해당하는 경우가 발생하였을 때, 그 사실을 안 날부터 1월 이내에 보험계약을 해지할 수 있습니다. 다만, 제1호·제2호·제4호·제5호에 의한 계약해지는 의무보험에 대해 적용하지 않습니다. 1. 보험계약자가 보험계약을 맺을 때 고의 또는 중대한 과실１２９)로 제44조(계약 전 알릴 의무) 제1항의 사항에 관하여 알 고 있는 사실을 알리지 않거나 사실과 다르게 알린 경우. 다만, 다음 중 어느 하나에 해당하는 경우 보험회사는 보험 계약을 해지하지 못합니다. 가. 보험계약을 맺은 때에 보험회사가 보험계약자가 알려야 할 사실을 알고 있었거나 과실로 알지 못하였을 때 나. 보험계약자가 보험금을 지급할 사고가 발생하기 전에 보험청약서의 기재사항에 대하여 서면으로 변경을 신청하 여 보험회사가 이를 승인하였을 때 다. 보험회사가 보험계약을 맺은 날부터 보험계약을 해지하지 않고 6개월이 경과한 때 라. 보험을 모집한 자(이하 ‘보험설계사 등’이라 합니다)가 보험계약자 또는 피보험자에게 계약 전 알릴 의무를 이행할 기회를 부여하지 아니하였거나 보험계약자 또는 피보험자가 사실대로 알리는 것을 방해한 경우, 또는 보험계약자 또는 피보험자에 대해 사실대로 알리지 않게 하였거나 부실하게 알리도록 권유했을 때. 다만, 보험설계사 등의 행 위가 없었다 하더라도 보험계약자 또는 피보험자가 사실대로 알리지 않

In [23]:
import torch
from transformers import PhiForCausalLM, AutoModelForCausalLM, AutoTokenizer

torch.set_default_device("cpu")

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained("kakaocorp/kanana-nano-2.1b-base", torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained("kakaocorp/kanana-nano-2.1b-base", trust_remote_code=True)

system_message = "당신은 보험 상담 ASSISTANT입니다. 보험 약관인 context 기반으로 query에 대해 답변해 주세요."
prompt = f"[SYSTEM]: {system_message}\n[USER]: {query}\n[context]: {context}\n[ASSISTANT]"

inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_new_tokens=512, max_length=2048)
text = tokenizer.batch_decode(outputs)[0]
print(text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


<|begin_of_text|>[SYSTEM]: 당신은 보험 상담 ASSISTANT입니다. 보험 약관인 context 기반으로 query에 대해 답변해 주세요.
[USER]: 의무보험 미가입에 따른 불이익을 알려줘
[context]: ['않는 경우 해당 계약을 해지할 수 있습니다. ④ 제1항 및 제3항에 따라 계약이 해지된 경우 회사는 제54조(보험료의 환급) 제3항 제1호에 따른 보험료를 계약자에게 지 급합니다. ⑤ 계약자는 제1항에 따른 제척기간１２８)에도 불구하고 민법 등 관계 법령에서 정하는 바에 따라 법률상의 권리를 행사할 수 있습니다. 제53조(보험회사의 보험계약 해지) ① 보험회사는 다음 중 어느 하나에 해당하는 경우가 발생하였을 때, 그 사실을 안 날부터 1월 이내에 보험계약을 해지할 수 있습니다. 다만, 제1호·제2호·제4호·제5호에 의한 계약해지는 의무보험에 대해 적용하지 않습니다. 1. 보험계약자가 보험계약을 맺을 때 고의 또는 중대한 과실１２９)로 제44조(계약 전 알릴 의무) 제1항의 사항에 관하여 알 고 있는 사실을 알리지 않거나 사실과 다르게 알린 경우. 다만, 다음 중 어느 하나에 해당하는 경우 보험회사는 보험 계약을 해지하지 못합니다. 가. 보험계약을 맺은 때에 보험회사가 보험계약자가 알려야 할 사실을 알고 있었거나 과실로 알지 못하였을 때 나. 보험계약자가 보험금을 지급할 사고가 발생하기 전에 보험청약서의 기재사항에 대하여 서면으로 변경을 신청하 여 보험회사가 이를 승인하였을 때 다. 보험회사가 보험계약을 맺은 날부터 보험계약을 해지하지 않고 6개월이 경과한 때 라. 보험을 모집한 자(이하 ‘보험설계사 등’이라 합니다)가 보험계약자 또는 피보험자에게 계약 전 알릴 의무를 이행할 기회를 부여하지 아니하였거나 보험계약자 또는 피보험자가 사실대로 알리는 것을 방해한 경우, 또는 보험계약자 또는 피보험자에 대해 사실대로 알리지 않게 하였거나 부실하게 알리도록 권유했을 때. 다만, 보험설계사 등의 행 위가 없었다 하더라도 보험계약자 또는

In [ ]:
import faiss
import pickle
import numpy as np
from langchain.schema import Document
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoModel, AutoTokenizer
import torch

model_name = "kakaocorp/kanana-nano-2.1b-embedding"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)

def get_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=2000, return_tensors="pt")

    # pool_mask 생성 (입력 시퀀스 길이에 맞춰서 1로 설정)
    pool_mask = torch.ones(inputs["input_ids"].shape, dtype=torch.long)

    with torch.no_grad():
        outputs = model(**inputs, pool_mask=pool_mask)  # pool_mask 추가

    return outputs.embedding.numpy()

In [11]:
from langchain.vectorstores import FAISS
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
import pickle
import os
from transformers import AutoModel, AutoTokenizer
import torch

model_name = "kakaocorp/kanana-nano-2.1b-embedding"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)

def get_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=2000, return_tensors="pt")

    # pool_mask 생성 (입력 시퀀스 길이에 맞춰서 1로 설정)
    pool_mask = torch.ones(inputs["input_ids"].shape, dtype=torch.long)

    with torch.no_grad():
        outputs = model(**inputs, pool_mask=pool_mask)  # pool_mask 추가


# 1️⃣ FAISS 저장 경로
faiss_index_path = "./faiss_index1"
metadata_path = "./documents1.pkl"

# 2️⃣ 임베딩 모델 설정
# embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 3️⃣ 문서 리스트 가져오기
documents = []

# 4️⃣ PDF 문서 로드 & 전처리
for file in file_list:  # 상위 3개 파일만 처리
    loader = PyPDFLoader(path + "\\" + file)
    docs = loader.load()

    # 줄바꿈 및 공백 정리 + 메타데이터 추가
    for doc in docs:
        doc.page_content = doc.page_content.replace("\n", " ").replace("  ", " ")
        doc.metadata["source"] = file

    # 5️⃣ 문서 스플리팅
    splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=1000)
    split_docs = splitter.split_documents(docs)

    # 문서 저장
    documents.extend(split_docs)

    # 6️⃣ FAISS 벡터스토어 생성 & 저장
    faiss_index = FAISS.from_documents(documents, embedding_model)
    faiss_index.save_local(faiss_index_path)  # FAISS 저장

    # 7️⃣ 문서 정보 저장
    with open(metadata_path, "ab") as f:
        pickle.dump(documents, f)

    print(file)


KeyboardInterrupt: 

In [ ]:
# !pip install fitz
# !pip install datasets
# !pip install frontend
# !pip install pymupdf

In [ ]:
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoModel, AutoTokenizer
import torch
import pickle
import fitz  # PyMuPDF

# 1️⃣ 저장 경로 설정
faiss_index_path = "./faiss_index"
metadata_path = "./documents.pkl"
pdf_path = "./pdf_folder"  # PDF 파일 경로

# 2️⃣ Hugging Face 임베딩 모델 로드
model_name = "kakaocorp/kanana-nano-2.1b-embedding"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)

def get_embeddings(texts):
    """주어진 텍스트 리스트를 임베딩 벡터로 변환"""
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=1000)
    
    pool_mask = torch.ones(inputs["input_ids"].shape, dtype=torch.float)

    with torch.no_grad():
        outputs = model(**inputs, pool_mask=pool_mask)
    return outputs.last_hidden_state[:, 0, :].numpy()  # CLS 토큰 벡터 사용

# 3️⃣ 문서 리스트 가져오기
all_documents = []  # 전체 문서를 저장할 리스트
batch_size = 10
# 4️⃣ PDF 문서 로드 & 전처리
for file in file_list:
    documents = []  # 현재 파일의 문서 리스트
    file_path = path + '\\' + file
    try:
        loader = PyPDFLoader(file_path)
        documents = loader.load()
    except:
        documents = []
        file_path = path + '\\' + file
        with fitz.open(file_path) as pdf:
            metadata = pdf.metadata # PDF의 메타데이터 추출
            
            for page_num, page in enumerate(pdf):
                page_text = page.get_text()
                if "MuPDF error:" not in page_text:
                    documents.append(Document(
                        page_content=page_text,
                        metadata={
                            'producer': metadata.get('producer', ''),
                            'creator': metadata.get('creator', ''),
                            'creationdate': metadata.get('creationDate', ''),
                            'title': metadata.get('title', ''),
                            'author': metadata.get('author', ''),
                            'moddate': metadata.get('modDate', ''),
                            'pdfversion': metadata.get('pdfVersion', ''),
                            'source': file_path,
                            'total_pages': pdf.page_count,
                            'page': page_num,
                            'page_label': str(page_num + 1)
                        }
                    ))
    # 줄바꿈 및 공백 정리 + 메타데이터 추가
    for doc in documents:
        doc.page_content = doc.page_content.replace("\n", " ").replace("  ", " ")
        doc.metadata["source"] = file

    # 5️⃣ 문서 스플리팅
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=500)
    split_docs = splitter.split_documents(documents)

    # ✅ 문서 리스트에 추가
    all_documents.extend(split_docs)
    # print(file)

    # 6️⃣ 문서 임베딩 및 FAISS 벡터스토어 생성
    texts = [doc.page_content for doc in all_documents]
    embeddings = get_embeddings(texts)  # ⚡ KANANA 모델로 벡터 변환
    faiss_index.add_vectors(embeddings, split_docs)

    all_documents.extend(split_docs)
    print(f"✅ {file} 처리 완료!")
# faiss_index = FAISS.from_embeddings(embeddings, documents)
faiss_index.save_local(faiss_index_path)  # FAISS 저장

# 7️⃣ 문서 정보 저장 (이전 데이터 유지)
try:
    with open(metadata_path, "rb") as f:
        existing_documents = pickle.load(f)
except (FileNotFoundError, EOFError):
    existing_documents = []

existing_documents.extend(documents)

with open(metadata_path, "wb") as f:
    pickle.dump(existing_documents, f)



c:\Users\kwj\anaconda3\envs\vectordb\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: [enforce fail at alloc_cpu.cpp:115] data. DefaultCPUAllocator: not enough memory: you tried to allocate 13114450944 bytes.